In [3]:
import re, collections
from textblob import TextBlob
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.base import TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sklearn.feature_extraction.stop_words import ENGLISH_STOP_WORDS
from sklearn.metrics import accuracy_score
from nltk.corpus import stopwords
from spacy.en import English ##Note you'll need to install Spacy and download its dependencies
parser = English()
import string

In [5]:
# A custom stoplist
STOPLIST = set(stopwords.words('english') + ["n't", "'s", "'m", "ca"] + list(ENGLISH_STOP_WORDS))
# List of symbols we don't care about
SYMBOLS = " ".join(string.punctuation).split(" ") + ["-----", "---", "...", "“", "”", "'ve"]

In [13]:
# A custom function to clean the text before sending it into the vectorizer
def cleanText(text):
    # get rid of newlines
    text = text.strip().replace("\n", " ").replace("\r", " ")
    
    # replace twitter @mentions
    mentionFinder = re.compile(r"@[a-z0-9_]{1,15}", re.IGNORECASE)
    text = mentionFinder.sub("@MENTION", text)
    text = re.sub('[^a-zA-Z ]','',text)
    # replace HTML symbols
    text = text.replace("&amp;", "and").replace("&gt;", ">").replace("&lt;", "<")
    
    # lowercase
    text = text.lower()
#     text = str(TextBlob(text).correct())
    return text

# A custom function to tokenize the text using spaCy
# and convert to lemmas
def tokenizeText(sample):

    # get the tokens using spaCy
    tokens = parser(sample)

    # lemmatize
    lemmas = []
    for tok in tokens:
        lemmas.append(tok.lemma_.lower().strip() if tok.lemma_ != "-PRON-" else tok.lower_)
    tokens = lemmas

    # stoplist the tokens
    tokens = [tok for tok in tokens if tok not in STOPLIST]

    # stoplist symbols
    tokens = [tok for tok in tokens if tok not in SYMBOLS]

    # remove large strings of whitespace
    while "" in tokens:
        tokens.remove("")
    while " " in tokens:
        tokens.remove(" ")
    while "\n" in tokens:
        tokens.remove("\n")
    while "\n\n" in tokens:
        tokens.remove("\n\n")

    return tokens

In [7]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = CountVectorizer(ngram_range=(1,3),min_df=3, max_features=1000,tokenizer=tokenizeText)
tfvectorizer = TfidfVectorizer(ngram_range=(1,3),min_df = 3,max_features=1000,tokenizer=tokenizeText,sublinear_tf=True)

### Take note of the assumptions made in the vectorizer specifications. There are two kinds of vectorizers initialized: count and tfidf. I've limited both to consider only n-grams that appear at least ten times. I've also limited the feature set into the top 1,000 n-grams that appear the most often in the reviews. Also it only extracts unigrams to trigrams. You can edit any of the parameters.

### You can switch between count and tfidf vectorizers by changing between "vectorizer" and "tfvectorizer" in one of the cells below

In [8]:
import pandas as pd
import pickle
with open("C:/Users/kenndanielso/Documents/Github/mcnulty_yelp/data/review_business_df.pkl", 'rb') as picklefile: 
    review_business_df = pickle.load(picklefile)

In [9]:
sample_df = review_business_df.sample(10000,random_state=1).dropna().reset_index(drop=True)

### Note that I've only extract 10,000 sample reviews. This is just to manage its tracktability. Lemmatizing takes a really long time. Indirectly, 10,000 reviews is also big enough relative to the 1,000 features (discussed above).

In [14]:
short_df = sample_df.iloc[:,0:2]
short_df.text= short_df.text.apply(cleanText)

In [15]:
## Gets the count of each word in each sentence
countfeature = vectorizer.fit_transform(short_df.text)
tffeature = tfvectorizer.fit_transform(short_df.text)

In [16]:
## Turns count/tfidf matrix into a dataframe
countfeaturedf = pd.DataFrame(countfeature.A, columns=vectorizer.get_feature_names())
tffeaturedf = pd.DataFrame(tffeature.A, columns=tfvectorizer.get_feature_names())

In [19]:
## Concat Y and X
new_df_count = pd.concat((short_df,countfeaturedf),axis=1)
new_df_tf = pd.concat((short_df,tffeaturedf),axis=1)

In [20]:
print(new_df_count.info())
print(new_df_tf.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Columns: 1002 entries, text to yummy
dtypes: int64(1001), object(1)
memory usage: 76.4+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Columns: 1002 entries, text to yummy
dtypes: float64(1000), int64(1), object(1)
memory usage: 76.4+ MB
None


In [23]:
import pickle
new_df_count.to_pickle('new_df_count.pkl')
new_df_tf.to_pickle('new_df_tf.pkl')

In [24]:
def string(x):
    if x == 5:
        return "five"
    elif x == 4:
        return 'four'
    elif x == 3:
        return 'three'
    elif x == 2:
        return 'two'
    elif x == 1:
        return 'one'

In [25]:
new_df_count['stars_x'] = short_df['stars_x'].apply(string)
new_df_tf['stars_x'] = short_df['stars_x'].apply(string)

In [27]:
collapse_df_count = new_df_count.copy()
collapse_df_tf = new_df_tf.copy()

In [29]:
##Apply this if you want to collapse the 5-star ratings by removing 3-star reviews and combining 1 & 2 stars 
##and 4 & 5 ratings

def collapse(x):
    if x == 'four' or x == 'five':
        return 'four/five'
    elif x == 'one' or x == 'two':
        return 'one/two'
    elif x == 'three':
        return 'three'

collapse_df_count.stars_x = new_df_count.stars_x.apply(collapse)
collapse_df_tf.stars_x = new_df_tf.stars_x.apply(collapse)

In [30]:
# Run if you want to delete 3-stars
# collapse_df = collapse_df[new_df['stars_x'] != 'three']

In [33]:
##Adds sentiment as a feature. Note that I added 1 because some algorithms won't accept negative sentiment scores
##Sentiment scores is based on TextBlob where it goes from -1.0 to 1.0 (negative to positive)
##Change the reference DF if you want to go back to using the 5-star categories

from textblob import TextBlob

collapse_df_count['senti'] = collapse_df_count['text'].apply(lambda x: TextBlob(x).sentiment[0] + 1)
collapse_df_tf['senti'] = collapse_df_tf['text'].apply(lambda x: TextBlob(x).sentiment[0] + 1)


# Model testing

In [65]:
##Split into train and test at 75/25
##Change the reference DF if you want to go back to using the 5-star categories
from sklearn.cross_validation import train_test_split 
train, test = train_test_split(collapse_df_count,test_size = 0.25)

In [66]:
##Split X & Y
X_train = train.iloc[:,2:]
Y_train = train.iloc[:,1]
X_test = test.iloc[:,2:]
Y_test = test.iloc[:,1]

In [67]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

In [68]:
## KNN
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=1).fit(X_train,Y_train)
knn_Y_pred = knn.predict(X_test)
print("KNN Accuracy: ",np.mean(knn_Y_pred == np.array(Y_test)))
print(classification_report(Y_test,knn_Y_pred))

KNN Accuracy:  0.642
             precision    recall  f1-score   support

  four/five       0.74      0.83      0.78      1633
    one/two       0.44      0.43      0.43       475
      three       0.24      0.13      0.17       392

avg / total       0.61      0.64      0.62      2500



In [69]:
## NaiveBayes
from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB().fit(X_train,Y_train)
nb_Y_pred = nb.predict(X_test)
print("NB Accuracy: ",np.mean(nb_Y_pred == np.array(Y_test)))
print(classification_report(Y_test,nb_Y_pred))

NB Accuracy:  0.7584
             precision    recall  f1-score   support

  four/five       0.86      0.87      0.87      1633
    one/two       0.69      0.67      0.68       475
      three       0.40      0.39      0.40       392

avg / total       0.76      0.76      0.76      2500



In [70]:
## Logistic
from sklearn.linear_model import LogisticRegression
log = LogisticRegression().fit(X_train,Y_train)
log_Y_pred = log.predict(X_test)
print("Logistc Accuracy: ",np.mean(log_Y_pred == np.array(Y_test)))
print(classification_report(Y_test,log_Y_pred))

Logistc Accuracy:  0.778
             precision    recall  f1-score   support

  four/five       0.84      0.93      0.88      1633
    one/two       0.68      0.72      0.70       475
      three       0.45      0.23      0.30       392

avg / total       0.75      0.78      0.76      2500



In [71]:
## Linear SVC
from sklearn.svm import SVC
svcl = SVC(kernel='linear').fit(X_train,Y_train)
svcl_Y_pred = svcl.predict(X_test)
print("SVC Linear Accuracy: ",np.mean(svcl_Y_pred == np.array(Y_test)))
print(classification_report(Y_test,svcl_Y_pred))

SVC Linear Accuracy:  0.7556
             precision    recall  f1-score   support

  four/five       0.84      0.90      0.87      1633
    one/two       0.66      0.67      0.67       475
      three       0.37      0.26      0.30       392

avg / total       0.73      0.76      0.74      2500



In [72]:
## RBF SVC
from sklearn.svm import SVC
svcrbf = SVC(kernel='rbf', gamma=1).fit(X_train,Y_train)
svcrbf_Y_pred = svcrbf.predict(X_test)
print("SVC RBF Accuracy: ",np.mean(svcrbf_Y_pred == np.array(Y_test)))
print(classification_report(Y_test,svcrbf_Y_pred))

KeyboardInterrupt: 

In [ ]:
## Decision Tree
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier().fit(X_train,Y_train)
dt_Y_pred = dt.predict(X_test)
print("Decision Tree Accuracy: ",np.mean(dt_Y_pred == np.array(Y_test)))
print(classification_report(Y_test,dt_Y_pred))

In [ ]:
## Random Forests
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier().fit(X_train,Y_train)
rf_Y_pred = rf.predict(X_test)
print("Random Forests Accuracy: ",np.mean(rf_Y_pred == np.array(Y_test)))
print(classification_report(Y_test,rf_Y_pred))

# Read twitter feeds

In [133]:
tweet_df = pd.read_pickle('C:/Users/kenndanielso/Documents/Github/mcnulty_yelp/data/tweets_clean.pkl')
tweet_df.reset_index(drop=True,inplace=True)

In [134]:
tweet_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 803 entries, 0 to 802
Data columns (total 6 columns):
business      803 non-null object
created_at    803 non-null object
lang          803 non-null object
location      527 non-null object
user_id       803 non-null int64
text          803 non-null object
dtypes: int64(1), object(5)
memory usage: 37.7+ KB


In [135]:
tweet_sparse_matrix = vectorizer.transform(tweet_df.text)

In [136]:
tweet_count_df = pd.DataFrame(tweet_sparse_matrix.A, columns=vectorizer.get_feature_names())

In [137]:
import pandas as pd
tweet_df2 = pd.concat((tweet_df,tweet_count_df),axis=1)
tweet_df2['senti'] = tweet_df.text.apply(lambda x: TextBlob(x).sentiment[0] + 1)

In [138]:
X_tweet = tweet_df2.iloc[:,6:]

In [139]:
## Naive Bayes
tweet_pred = nb.predict(X_tweet)

In [140]:
tweet_df['pred'] = tweet_pred

In [142]:
tweet_df.to_pickle("tweet_df_pred.pkl")